<h1>Biblioteca</h1>

In [1]:
import pandas as pd
import numpy as np 




<h1>Pré-Processamento</h1>

In [2]:
## Dataframe Origenal
df = pd.read_json('https://jeap.rio.rj.gov.br/dadosAbertosAPI/v2/transporte/veiculos/onibus2')
df

,ordem,linha,latitude,longitude,velocidade,dataHora
0,A29139,583,-22.86854,-43.29169,0,05-25-2022 05:13:32
1,D87879,2336,-22.91699,-43.60768,0,05-25-2022 07:07:39
2,D13062,745,-22.84873,-43.54393,30,05-25-2022 07:11:44
3,B32716,946,-22.81725,-43.39347,0,05-25-2022 07:45:40
4,B71151,247,-22.90448,-43.29535,7,05-25-2022 08:29:08
...,...,...,...,...,...,...
3478,B11562,665,-22.81769,-43.30705,0,05-25-2022 18:21:59
3479,B11639,363,-22.88409,-43.35703,0,05-25-2022 18:21:59
3480,B71032,455,-22.92383,-43.17087,57,05-25-2022 18:21:59
3481,B11548,363,-22.88294,-43.36746,0,05-25-2022 18:22:00


In [3]:
df.columns

Index(['ordem', 'linha', 'latitude', 'longitude', 'velocidade', 'dataHora'], dtype='object')

In [4]:
##Exportação do dataframe em Python

##df_excel=df
##df_excel.to_excel (r'C:\Users\Gabriel\Desktop\Mestrado\export_dataframe.xlsx', index = False, header=True)

In [14]:
#OrdemOnibus = df['ordem'].unique()
#ArrayOrdemOnibus = pd.DataFrame(OrdemOnibus)
#print(ArrayOrdemOnibus)

#NOVO DATAFRAME
ordem=df['ordem']
ordemslice=df['ordem'].str.slice(1,4)

df2= pd.DataFrame(list(zip(ordem,ordemslice)),columns=['OrdemOnibus','CodEmpr'])


# The scope of these changes made to
# pandas settings are local to with statement.
with pd.option_context('display.max_rows', None,
                       'display.max_columns', None,
                       'display.precision', 3,
                       ):
    print(df2)



     OrdemOnibus CodEmpr
0         A29139     291
1         D87879     878
2         D13062     130
3         B32716     327
4         B71151     711
5         D17047     170
6         D17047     170
7         B27077     270
8         D13290     132
9         C47663     476
10        C30042     300
11        A41312     413
12        A41447     414
13        D17191     171
14        D17191     171
15        B32624     326
16        D17180     171
17        B32668     326
18        B63032     630
19        A29089     290
20        D17075     170
21        D17075     170
22        D13291     132
23        A63531     635
24        A71583     715
25        A71583     715
26        A29067     290
27        A29019     290
28        D13298     132
29        A29156     291
30        B32675     326
31        A29084     290
32        B63078     630
33        D86181     861
34        C47576     475
35        C30373     303
36        A41218     412
37        D13089     130
38        D13511     135
